# This uses all four datasets

### Setup

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
%matplotlib inline

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Check GPU Avaliability

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### Data

#### Loading Data

In [ ]:
dataset1 = pd.read_csv('/content/drive/My Drive/184A/finalProj/data/processed.cleveland.txt', header = None)
dataset1.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num']
dataset1 = dataset1.replace('?', np.nan)
dataset1 = dataset1.astype({'age': 'float64','sex': 'category','cp': 'category','trestbps': 'float64','chol': 'float64', 'fbs': 'category', 'restecg': 'category', 'thalach': 'float64', 'exang': 'category', 'oldpeak': 'float64', 'slope': 'category', 'ca': 'category', 'thal': 'category', 'num': 'category'})

dataset2 = pd.read_csv('/content/drive/My Drive/184A/finalProj/data/reprocessed.hungarian.txt', sep=" ", header = None)
dataset2.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num']
dataset2 = dataset2.replace(-9, np.nan)
dataset2['ca'] = dataset2['ca'].replace(9, np.nan)
dataset2 = dataset2.astype({'age': 'float64','sex': 'category','cp': 'category','trestbps': 'float64','chol': 'float64', 'fbs': 'category', 'restecg': 'category', 'thalach': 'float64', 'exang': 'category', 'oldpeak': 'float64', 'slope': 'category', 'ca': 'category', 'thal': 'category', 'num': 'category'})

dataset3 = pd.read_csv('/content/drive/My Drive/184A/finalProj/data/processed.switzerland.txt', header = None)
dataset3.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num']
dataset3 = dataset3.replace('?', np.nan)
dataset3['chol'] = dataset3['chol'].replace(0,np.nan)
dataset3 = dataset3.astype({'age': 'float64','sex': 'category','cp': 'category','trestbps': 'float64','chol': 'float64', 'fbs': 'category', 'restecg': 'category', 'thalach': 'float64', 'exang': 'category', 'oldpeak': 'float64', 'slope': 'category', 'ca': 'category', 'thal': 'category', 'num': 'category'})

dataset4 = pd.read_csv('/content/drive/My Drive/184A/finalProj/data/processed.va.txt', header = None)
dataset4.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num']
dataset4 = dataset4.replace('?', np.nan)
dataset4['chol'] = dataset4['chol'].replace('0',np.nan)
dataset4['trestbps'] = dataset4['trestbps'].replace('0',np.nan)
dataset4 = dataset4.astype({'age': 'float64','sex': 'category','cp': 'category','trestbps': 'float64','chol': 'float64', 'fbs': 'category', 'restecg': 'category', 'thalach': 'float64', 'exang': 'category', 'oldpeak': 'float64', 'slope': 'category', 'ca': 'category', 'thal': 'category', 'num': 'category'})

#### Creating Datasets

In [ ]:
datasetAll = pd.concat([dataset1, dataset2, dataset3, dataset4], ignore_index=True, sort=False)

#top 6 features - oldpeak, chol, thalach, age, trestbps, ca
#middle - slope
#bottom - restecg, cp, thal, fbs, exang, sex (lowest)

#drop ca, slope, thal
#(558, 11)
datasetV1 = datasetAll.drop(['slope', 'ca', 'thal'], axis = 1)
datasetV1 = datasetV1.dropna()
datasetV1 = datasetV1.reset_index(drop=True)

#drop ca, thal (will lose 1/3 of data since slope is 1/3 nan)
#(392, 12)
datasetV2 = datasetAll.drop(['ca', 'thal'], axis = 1)
datasetV2 = datasetV2.dropna()
datasetV2 = datasetV2.reset_index(drop=True)

#drop thal (will still lose majority of data since ca had 2/3 nan)
#(297, 13)
datasetV3 = datasetAll.drop(['thal'], axis = 1)
datasetV3 = datasetV3.dropna()
datasetV3 = datasetV3.reset_index(drop=True)

print(datasetV1.shape)
print(datasetV1['num'].value_counts())
print()
print(datasetV2.shape)
print(datasetV2['num'].value_counts())
print()
print(datasetV3.shape)
print(datasetV3['num'].value_counts())

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#print(datasetV1.shape)

(661, 11)
0.0    347
1.0    119
3.0     82
2.0     80
4.0     33
Name: num, dtype: int64

(462, 12)
0.0    201
1.0     94
3.0     73
2.0     64
4.0     30
Name: num, dtype: int64

(301, 13)
0.0    161
1.0     56
2.0     36
3.0     35
4.0     13
Name: num, dtype: int64


#### Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
datasetV1.iloc[:,0:-1] = scaler.fit_transform(datasetV1.iloc[:,0:-1])
datasetV2.iloc[:,0:-1] = scaler.fit_transform(datasetV2.iloc[:,0:-1])
datasetV3.iloc[:,0:-1] = scaler.fit_transform(datasetV3.iloc[:,0:-1])

### Choosing dataset

In [ ]:
theDataset = datasetV1 #10 inputs w/o ca, slope, thal
#theDataset = datasetV2 #11 inputs w/o ca, thal
#theDataset = datasetV3 #12 inputs w/o thal
inputNum = theDataset.shape[1] -1
print(theDataset.shape)
print(theDataset['num'].value_counts())

(661, 11)
0.0    347
1.0    119
3.0     82
2.0     80
4.0     33
Name: num, dtype: int64


#### To turn it into binary classifier

In [ ]:
#def changeTarget(value):
#  return 1 if value > 0 else 0
#theDataset['num'] = theDataset['num'].apply(changeTarget) #to make it just 0 and 1's

### Hyper-parameters

In [ ]:
# Hyper-parameters
input_size = inputNum
hidden_size = inputNum-1
num_classes = (theDataset['num'].value_counts()).size

num_epochs = 100
batch_size = 30
learning_rate = 0.001

### Setting Up Data

#### Seperate

In [ ]:
#from sklearn.utils import shuffle
#np.random.seed(0)
#theDataset = shuffle(theDataset)

#X = theDataset.iloc[:,0:-1]
#y = theDataset.iloc[:,-1]
#Xtr, Xva, Ytr, Yva = train_test_split(X, y, test_size=0.25, random_state=2, stratify=y)

#### Seperate and Over/Undersampling

In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

from collections import Counter
from imblearn.under_sampling import NeighbourhoodCleaningRule, TomekLinks, ClusterCentroids, NearMiss, EditedNearestNeighbours, CondensedNearestNeighbour, OneSidedSelection, RandomUnderSampler
from imblearn.over_sampling import SMOTE

XtrSamp = Xtr
YtrSamp = Ytr

print('Original dataset shape %s' % Counter(Ytr))

print('\nOversampling')
smote = SMOTE()
X_res, y_res = smote.fit_resample(Xtr, Ytr)
print('SMOTE shape %s' % Counter(y_res))
#Xtr = X_res
#Ytr = y_res
#print("Doing Over")

print('\nRandom')
rus = RandomUnderSampler(sampling_strategy='auto')
X_res, y_res = rus.fit_resample(Xtr, Ytr)
print('RandomUnderSampler shape %s' % Counter(y_res))

print('\nKeep')
cnn = CondensedNearestNeighbour(sampling_strategy='auto')
X_res, y_res = cnn.fit_resample(Xtr, Ytr)
print('CondensedNearestNeighbour shape %s' % Counter(y_res))

nm = NearMiss(sampling_strategy='auto')
X_res, y_res = nm.fit_resample(Xtr, Ytr)
print('NearMiss shape %s' % Counter(y_res))

print('\nDelete')
tl = TomekLinks(sampling_strategy='auto')
X_res, y_res = tl.fit_resample(Xtr, Ytr)
print('TomekLinks shape %s' % Counter(y_res))

enn = EditedNearestNeighbours(sampling_strategy='majority')
X_res, y_res = enn.fit_resample(Xtr, Ytr)
print('EditedNearestNeighbours shape %s' % Counter(y_res))

print('\nBoth')
ncr = NeighbourhoodCleaningRule(sampling_strategy='auto')
X_res, y_res = ncr.fit_resample(Xtr, Ytr)
print('NeighbourhoodCleaningRule shape %s' % Counter(y_res))

oss = OneSidedSelection(sampling_strategy='auto')
X_res, y_res = oss.fit_resample(Xtr, Ytr)
print('OneSidedSelection shape %s' % Counter(y_res))

print()

cc = ClusterCentroids(sampling_strategy='auto')
X_res, y_res = cc.fit_resample(Xtr, Ytr)
print('ClusterCentroids shape %s' % Counter(y_res))

#XtrSamp = X_res
#YtrSamp = y_res
#print("Doing Above")

print()
dataSamp = pd.DataFrame(np.column_stack((XtrSamp,YtrSamp)))
dataSamp.rename({(inputNum): 'num'}, axis=1, inplace=True)
dataSamp = shuffle(dataSamp)
Xtr = dataSamp.iloc[:,0:-1]
Ytr = dataSamp.iloc[:,-1]

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#print(dataSamp)
print(dataSamp.shape)
print(dataSamp['num'].value_counts())

Original dataset shape Counter({0: 260, 1: 235})

Oversampling
SMOTE shape Counter({1: 260, 0: 260})

Random
RandomUnderSampler shape Counter({0: 235, 1: 235})

Keep
CondensedNearestNeighbour shape Counter({1: 235, 0: 90})
NearMiss shape Counter({0: 235, 1: 235})

Delete
TomekLinks shape Counter({1: 235, 0: 231})
EditedNearestNeighbours shape Counter({1: 235, 0: 143})

Both
NeighbourhoodCleaningRule shape Counter({1: 235, 0: 137})
OneSidedSelection shape Counter({1: 235, 0: 230})

ClusterCentroids shape Counter({0: 235, 1: 235})

(495, 11)
0.0    260
1.0    235
Name: num, dtype: int64


#### Into DataLoader

In [ ]:
pdXtr = Xtr
pdYtr = Ytr
pdXva = Xva
pdYva = Yva

npXtr = np.array(pdXtr, dtype='float')
npYtr = np.array(pdYtr, dtype='float')
npXva = np.array(pdXva, dtype='float')
npYva = np.array(pdYva, dtype='float')

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

tensorXtr = torch.Tensor(npXtr) # transform to torch tensor
tensorYtr = torch.Tensor(npYtr)
tensorYtr = tensorYtr.type(torch.LongTensor)
datasetTr = TensorDataset(tensorXtr, tensorYtr) # create your datset
trainloader = DataLoader(datasetTr, batch_size, True) # create your dataloader

tensorXva = torch.Tensor(npXva) # transform to torch tensor
tensorYva = torch.Tensor(npYva)
tensorYva = tensorYva.type(torch.LongTensor)
datasetVa = TensorDataset(tensorXva, tensorYva) # create your datset
valiloader = DataLoader(datasetVa, batch_size, True) # create your dataloader

data, labels = next(iter(trainloader))
print(data.shape, labels.shape)

torch.Size([30, 10]) torch.Size([30])


### Model

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, hidden_size) #input and first hidden layer
        self.fc2 = nn.Linear(hidden_size, hidden_size) #hidden layer
        self.fc3 = nn.Linear(hidden_size, hidden_size) #hidden layer
        self.fc4 = nn.Linear(hidden_size, num_classes) #output

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Training

In [ ]:
# Train the model
total_step = len(trainloader)
for epoch in range(num_epochs):
  for i, (data, target) in enumerate(trainloader):
    # Move tensors to the configured device
    data = data.to(device)
    target = target.to(device)

    # Forward pass
    outputs = model(data)
    loss = criterion(outputs, target)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 5 == 0 and (i+1) % 4 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

### Test

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for data, labels in valiloader:
        data = data.to(device)
        labels = labels.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        #print("Predicted:", predicted, "\nLabel:", labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test daata: {} %'.format(100 * correct / total))

Accuracy of the network on the test daata: 78.91566265060241 %


In [ ]:
#!cat /content/drive/My\ Drive/184A/finalProj/data/processed.cleveland.txt